## Introduction
이번 실습에서는 좀 더 복잡한 데이터와 모델을 최적화해보도록 하겠습니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from time import time

seed = 2020
np.random.seed(seed)
tf.random.set_random_seed(seed) # for reproducibility

## Dataset info
cifar-10은 10가지 카테고리의 32x32 컬러(RGB) 이미지들로 구성되어 있으며 50,000장의 training data, 10,000장의 test data로 이루어져 있습니다.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.
y_train = y_train.astype(np.int64)
x_test = x_test.astype(np.float32) / 255.
y_test = y_test.astype(np.int64)

print(f"input shape: {x_train.shape}")
print(f"label shape: {y_train.shape}")

## Model 생성

In [ ]:
def create_model(optimizer='adam', init='glorot_uniform'):
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(32, 32, 3)),
        keras.layers.Dropout(0.5),
        # hidden layers
        keras.layers.Dense(1024,kernel_initializer=init),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1024,kernel_initializer=init),
        keras.layers.Activation('relu'),
        keras.layers.Dropout(0.2),
        # output layer
        keras.layers.Dense(10,kernel_initializer=init),
        keras.layers.Activation('softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [ ]:
model = create_model()
model.summary()
hist = model.fit(x_train, y_train, 
                 batch_size=1024, epochs=5,
                 validation_data=(x_test, y_test))

## Plot function

In [ ]:
%matplotlib inline
def plot_fn(labels, train_loss_histories, train_acc_histories, test_acc_histories):
    fig = plt.figure(figsize=(18,5))
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)
    for label, train_loss_history, train_acc_history, test_acc_history in zip(labels, train_loss_histories, train_acc_histories, test_acc_histories):
        ax1.plot(train_loss_history, label=str(label))
        ax2.plot(train_acc_history, label=str(label))
        ax3.plot(test_acc_history, label=str(label))
        
    ax1.set_xlabel('Batch #')
    ax1.set_ylabel('Training Loss [entropy]')
    ax2.set_xlabel('Batch #')
    ax2.set_ylabel('Training Accuracy')
    ax3.set_xlabel('Batch #')
    ax3.set_ylabel('Test Accuracy')
    plt.legend()
    plt.show
plot_fn(['CIFAR-10'], [hist.history['loss']], [hist.history['acc']], [hist.history['val_acc']])

## Hyperparameter Optimization
scikit learn의 model_selection 모듈을 사용하면 간단하게 하이퍼파라미터 서치를 수행할 수 있습니다.

GridSearchCV는 K-fold cross validation을 기반으로 주어진 parameter grid를 모두 탐색하면서 최적의 하이퍼 파라미터를 찾아냅니다.

RandomizedSearchCV는 parameter grid를 모두 탐색하지 않고 랜덤하게 뽑은 몇 가지 하이퍼 파라미터 조합에 대해서만 탐색을 수행합니다.

자세한 내용은 [Sklearn Docs](https://scikit-learn.org/stable/modules/grid_search.html)에서 찾아볼 수 있습니다.

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model) # wrapper

# opt parameter candidates
optimizers = ['sgd', 'adam']
init = ['glorot_uniform', 'normal']
epochs = np.array([5, 10])
batches = np.array([128, 256, 1024])
# model parameters
params={}

# parameter grid generation
param_grid = dict(optimizer=optimizers, init=init, nb_epoch=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5) # 5-fold cross validation

In [ ]:
# 각 hyperparameter에 따른 모델들을 학습합니다
start=time()
grid_result = grid.fit(x_train, y_train, **params)
end=time()

In [ ]:
# 탐색된 모델들의 성능을 비교합니다
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("total time:",end-start)

In [ ]:
# 가장 좋은 validation accuracy를 보인 하이퍼파라미터로 model을 create 하고 training data를 다시 학습합니다
# 위에서 보인 가장 좋은 Hyperparameter를 직접 입력해주어도 되고, grid_result.best_params_['**'] 이라는 함수를 사용해도 됩니다.

best_model = ##TODO : Your Code ##
best_hist = ##TODO : Your Code ##

plot_fn(['CIFAR-10'], [best_hist.history['loss']], [best_hist.history['acc']], [best_hist.history['val_acc']])